## **This Notebook will construct the nodes and edges that we will build our graph with**

Import relevant libraries and import raw data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/current_clean_spectrum.csv")

In [3]:
#sns.lineplot(data=df, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [4]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,41.863303
1,201210-1-00,294,400.91116,201210-1,Normal,41.803843
2,201210-1-00,295,401.59454,201210-1,Normal,41.741884
3,201210-1-00,296,402.27789,201210-1,Normal,41.677722
4,201210-1-00,297,402.96127,201210-1,Normal,41.611654
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


Set each WaveNumber as a column

In [5]:
pivoted_df = df.pivot(index='SpecID', columns='WaveNumber', values='Absorbance').reset_index()
pivoted_df.columns.name = None

statuses = df[['SpecID', 'Status']].drop_duplicates()

pivoted_df = pd.merge(pivoted_df, statuses, on='SpecID')
pivoted_df = pivoted_df.set_index('SpecID')

pivoted_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.3053,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,41.863303,41.803843,41.741884,41.677722,41.611654,41.543974,41.474980,41.404968,41.334234,41.263073,...,6.942749,6.280946,5.549559,4.745724,3.866578,2.909255,1.870891,0.748623,-0.460415,Normal
201210-1-01,46.314608,47.323684,48.299209,49.241395,50.150457,51.026608,51.870063,52.681035,53.459738,54.206386,...,6.322524,6.769011,7.280928,7.861246,8.512936,9.238972,10.042323,10.925962,11.892860,Normal
201210-1-02,118.159018,114.686240,111.563911,108.777452,106.312282,104.153823,102.287493,100.698715,99.372907,98.295491,...,-4.668874,-4.633601,-4.557349,-4.439365,-4.278894,-4.075180,-3.827470,-3.535010,-3.197043,Normal
201210-1-03,175.466997,174.846086,174.188020,173.498226,172.782129,172.045155,171.292728,170.530275,169.763222,168.996993,...,-11.220152,-10.801936,-10.349539,-9.864191,-9.347124,-8.799567,-8.222752,-7.617909,-6.986269,Normal
201210-1-04,111.814973,106.629998,101.867380,97.512673,93.551430,89.969205,86.751551,83.884023,81.352173,79.141556,...,-11.600625,-11.689508,-11.752441,-11.789205,-11.799583,-11.783357,-11.740310,-11.670224,-11.572882,Normal


In [6]:
status_df = df.groupby('SpecID')[['SurID', 'Status']].first()
# surface_and_status.to_csv("../../data/surface_and_status.csv")
status_df

,SurID,Status
SpecID,,
201210-1-00,201210-1,Normal
201210-1-01,201210-1,Normal
201210-1-02,201210-1,Normal
201210-1-03,201210-1,Normal
201210-1-04,201210-1,Normal
...,...,...
210526-3-45,210526-3,Hyperglycemia
210526-3-46,210526-3,Hyperglycemia
210526-3-47,210526-3,Hyperglycemia


>#### **Calculate an adjacency matrix using a Gaussian Kernel**

In [7]:
from scipy.spatial.distance import pdist, squareform

# Drop the 'Status' column as it is not numeric
X = pivoted_df.drop(columns=['Status'])

# Calculate pairwise Euclidean distances
distances = pdist(X.values, metric='euclidean')

# Convert the condensed distances to a square matrix form
distance_matrix = squareform(distances)

In [36]:
def gaussian_kernel(distances, sigma):
    return np.exp(-distances**2 / (2 * sigma**2))

# Sigma is the bandwidth parameter
sigma = 1000 # Adjust sigma as needed
kernel_matrix = gaussian_kernel(distance_matrix, sigma)

# Convert the kernel matrix to a DataFrame
kernel_df = pd.DataFrame(kernel_matrix, index=pivoted_df.index, columns=pivoted_df.index)

In [37]:
display(kernel_df.head())
kernel_df.shape

SpecID,201210-1-00,201210-1-01,201210-1-02,201210-1-03,201210-1-04,201210-1-05,201210-1-06,201210-1-07,201210-1-09,201210-1-10,...,210526-3-40,210526-3-41,210526-3-42,210526-3-43,210526-3-44,210526-3-45,210526-3-46,210526-3-47,210526-3-48,210526-3-49
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1.000000e+00,5.223281e-01,2.041993e-12,3.238707e-29,6.210645e-02,7.092606e-03,6.889311e-03,5.617053e-02,1.098191e-01,2.546363e-01,...,3.642365e-01,3.575612e-01,3.035297e-01,3.504342e-01,3.669956e-01,3.469070e-01,3.382363e-01,3.214902e-01,3.404585e-01,3.275357e-01
201210-1-01,5.223281e-01,1.000000e+00,7.080821e-11,3.561950e-27,1.524305e-01,2.646638e-02,1.894481e-02,7.880597e-02,1.248266e-01,1.881442e-01,...,1.463401e-01,1.510983e-01,1.186799e-01,1.241170e-01,1.223171e-01,1.099289e-01,1.064914e-01,9.433732e-02,1.029282e-01,9.798889e-02
201210-1-02,2.041993e-12,7.080821e-11,1.000000e+00,3.632634e-17,2.094772e-12,1.153994e-12,2.167129e-13,7.571365e-13,1.781165e-12,5.291329e-13,...,1.098815e-14,1.357268e-14,9.133141e-15,7.952386e-15,7.310936e-15,5.373381e-15,5.925506e-15,8.663486e-15,5.738708e-15,4.737228e-15
201210-1-03,3.238707e-29,3.561950e-27,3.632634e-17,1.000000e+00,3.915083e-28,8.888854e-28,1.207711e-27,9.591665e-28,1.502113e-27,4.834902e-29,...,1.838809e-31,3.574220e-31,1.034418e-31,5.786555e-32,4.024946e-32,2.096590e-32,1.882910e-32,2.521435e-32,2.251648e-32,1.693417e-32
201210-1-04,6.210645e-02,1.524305e-01,2.094772e-12,3.915083e-28,1.000000e+00,4.029496e-01,6.008697e-02,7.634647e-02,7.142855e-02,8.525467e-02,...,2.561879e-02,2.661733e-02,1.913160e-02,1.882642e-02,1.583793e-02,1.382123e-02,1.374915e-02,1.037739e-02,1.158891e-02,1.250222e-02


(3045, 3045)

**Construct the node and edge import files with the required Admin Import Headers**

In [38]:
relationship_df = kernel_df.reset_index()
relationship_df = relationship_df.melt(id_vars='SpecID', var_name=':END_ID', value_name='Weight:float')
relationship_df = relationship_df.rename(columns={'SpecID': ':START_ID'})
relationship_df[':TYPE'] = 'LINK'

Drop Relationships a node has with itself, along with repeat relationships.

In [39]:
relationship_df = relationship_df[relationship_df[":START_ID"] != relationship_df[":END_ID"]]
relationship_df = relationship_df[relationship_df[":START_ID"] < relationship_df[":END_ID"]]  # Drops redundant relationships
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,5.223281e-01,LINK
6090,201210-1-00,201210-1-02,2.041993e-12,LINK
6091,201210-1-01,201210-1-02,7.080821e-11,LINK
9135,201210-1-00,201210-1-03,3.238707e-29,LINK
9136,201210-1-01,201210-1-03,3.561950e-27,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,9.409605e-01,LINK
9272020,210526-3-45,210526-3-49,9.400599e-01,LINK
9272021,210526-3-46,210526-3-49,9.561006e-01,LINK
9272022,210526-3-47,210526-3-49,9.334067e-01,LINK


In [40]:
node_df = status_df.drop(columns=['SurID']).reset_index()
node_df.rename(columns={'SpecID':'SpecID:ID', 'Status':'Status:LABEL'}, inplace=True)
node_df

,SpecID:ID,Status:LABEL
0,201210-1-00,Normal
1,201210-1-01,Normal
2,201210-1-02,Normal
3,201210-1-03,Normal
4,201210-1-04,Normal
...,...,...
3040,210526-3-45,Hyperglycemia
3041,210526-3-46,Hyperglycemia
3042,210526-3-47,Hyperglycemia
3043,210526-3-48,Hyperglycemia


**Set an Environmental Variable as your working Neo4J directory for this to work.**

In [41]:
import os

Neo4j_Directory = os.environ['NEO4J_Directory']
print(Neo4j_Directory)

/home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a


Save the nodes and relationships files in the import folder.

In [42]:
node_df.to_csv(Neo4j_Directory + '/import/nodes.csv')
relationship_df.to_csv(Neo4j_Directory + '/import/relationships.csv')

In [43]:
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,5.223281e-01,LINK
6090,201210-1-00,201210-1-02,2.041993e-12,LINK
6091,201210-1-01,201210-1-02,7.080821e-11,LINK
9135,201210-1-00,201210-1-03,3.238707e-29,LINK
9136,201210-1-01,201210-1-03,3.561950e-27,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,9.409605e-01,LINK
9272020,210526-3-45,210526-3-49,9.400599e-01,LINK
9272021,210526-3-46,210526-3-49,9.561006e-01,LINK
9272022,210526-3-47,210526-3-49,9.334067e-01,LINK


-----

#### **This Section Creates the Graph in Neo4J then runs the Queries.**

In [44]:
from neo4j import GraphDatabase
import subprocess

**Run the admin import command to create the graph from the CSVs**

In [45]:
# Construct the command Linux
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv',
    '--overwrite-destination',  'neo4j'
]

#Windows
#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'
#result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data/databases/neo4j:\nNodes:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/nodes.csv\n\nRelationships:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 13.49GiB\n  Free machine memory: 336.5MiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 12\n  Configured max memory: 178.0MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'Weight' normalized from 'float' --> 'double' in /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\nImport 

Restart the database for the changes to take effect


In [46]:
command = [
    './bin/neo4j', 'restart'
]

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Restart successful")
    print(result.stdout)
else:
    print("Error during restart:")
    print(result.stderr)

Restart successful
b'Stopping Neo4j......... stopped.\nConfiguration file validation successful (with warnings).\nDirectories in use:\nhome:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a\nconfig:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/conf\nlogs:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/logs\nplugins:      /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/plugins\nimport:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import\ndata:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data\ncertificates: /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3f

Run and connect to the Neo4j Database

In [47]:
uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
password = os.environ['NEO4J_Password']           # Neo4J password'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [66]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

##### **Define the graph algorithms.**

In [67]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [68]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [69]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [70]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [71]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [72]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [73]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [74]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [75]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 2
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

##### **Execute the Graph Algorithms and Store the Results**

In [76]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['name'], how='left')

graph_metrics_df = merged_df.rename(columns={'name' : 'SpecID'})
graph_metrics_df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,201210-1-00,NaN,2.659152e+02,0.0,0.160914,68,15
1,201210-1-01,NaN,1.136585e+02,0.0,0.154678,68,15
2,201210-1-02,NaN,1.739286e-10,0.0,0.150000,68,15
3,201210-1-03,NaN,3.669687e-17,0.0,0.150000,68,15
4,201210-1-04,NaN,3.709101e+01,0.0,0.151536,47,802
...,...,...,...,...,...,...,...
3040,210526-3-45,NaN,6.049882e+02,0.0,0.174793,96,25
3041,210526-3-46,NaN,6.118620e+02,0.0,0.175073,96,25
3042,210526-3-47,NaN,5.968049e+02,0.0,0.174450,96,25
3043,210526-3-48,NaN,6.097860e+02,0.0,0.174985,96,25


##### **Get Node Embeddings using FastRP**

In [77]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1
0,201210-1-00,-1.998488,-0.074853
1,201210-1-01,-1.992880,0.109282
2,201210-1-02,-0.648716,1.194274
3,201210-1-03,1.014207,0.216980
4,201210-1-04,-1.565587,1.027828


**Delete the projection and close the session**

In [78]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)
    
close_driver()